In [1]:
import os
import sys
import json
import re
import socket
import ssl
import pandas as pd
import smtplib as smtp
import requests
from pandas.io.json import json_normalize
from urllib.request import Request, urlopen, URLError, HTTPError, ProxyHandler, build_opener, install_opener
from urllib.parse import quote, urlsplit, urlunsplit, urlencode
from random import randint
from time import sleep
from bs4 import BeautifulSoup

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 YaBrowser/19.6.1.153 Yowser/2.5 Safari/537.36'
PROXIES = {'https': 'https://37.203.246.48:42911'}
MIN_TIME_SLEEP = 1
MAX_TIME_SLEEP = 3
MAX_COUNTS = 5
TIMEOUT = 10
URL_BASE = 'https://www.pfl-russia.com'

In [3]:
CACHE_PATH = './_data/{}/'.format('pfl')
if not os.path.exists(CACHE_PATH):
    os.makedirs(CACHE_PATH)
CACHE_PATH_CLDS = '{}clds/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_CLDS):
    os.makedirs(CACHE_PATH_CLDS)
CACHE_PATH_CLDS_RAW = '{}clds_raw/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_CLDS_RAW):
    os.makedirs(CACHE_PATH_CLDS_RAW)
CACHE_PATH_CLDS_RFRS = '{}clds_rfrs/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_CLDS_RFRS):
    os.makedirs(CACHE_PATH_CLDS_RFRS)
CACHE_PATH_CLDS_RFRS_RAW = '{}clds_rfrs_raw/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_CLDS_RFRS_RAW):
    os.makedirs(CACHE_PATH_CLDS_RFRS_RAW)
CACHE_PATH_GMS = '{}gms/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_GMS):
    os.makedirs(CACHE_PATH_GMS)
CACHE_PATH_GMS_RAW = '{}gms_raw/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_GMS_RAW):
    os.makedirs(CACHE_PATH_GMS_RAW)
CACHE_PATH_TMS = '{}tms/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_TMS):
    os.makedirs(CACHE_PATH_TMS)
CACHE_PATH_TMS_RAW = '{}tms_raw/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_TMS_RAW):
    os.makedirs(CACHE_PATH_TMS_RAW)
CACHE_PATH_PLRS = '{}plrs/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_PLRS):
    os.makedirs(CACHE_PATH_PLRS)
CACHE_PATH_PLRS_RAW = '{}plrs_raw/'.format(CACHE_PATH)
if not os.path.exists(CACHE_PATH_PLRS_RAW):
    os.makedirs(CACHE_PATH_PLRS_RAW)

## Utils

In [4]:
def get_content(url_page, timeout, proxies, file=False):
    counts = 0
    content = None
    while counts < MAX_COUNTS:
        try:
            request = Request(url_page)
            request.add_header('User-Agent', USER_AGENT)
            proxy_support = ProxyHandler(proxies)
            opener = build_opener(proxy_support)
            install_opener(opener)
            context = ssl._create_unverified_context()
            response = urlopen(request, context=context, timeout=timeout)
            if file:
                content = response.read()
            else:
                content =  response.read().decode(response.headers.get_content_charset())
            break
        except URLError as e:
            counts += 1
            print('URLError | ', url_page, ' | ', e, ' | counts: ', counts)
            sleep(randint(counts * MIN_TIME_SLEEP, counts * MAX_TIME_SLEEP))
        except HTTPError as e:
            counts += 1
            print('HTTPError | ', url_page, ' | ', e, ' | counts: ', counts)
            sleep(randint(counts * MIN_TIME_SLEEP, counts * MAX_TIME_SLEEP))
        except socket.timeout as e:
            counts += 1
            print('socket timeout | ', url_page, ' | ', e, ' | counts: ', counts)
            sleep(randint(counts * MIN_TIME_SLEEP, counts * MAX_TIME_SLEEP))
    return content
def translit(text):
    symbols = ('абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ /-',
               'abvgdeejzijklmnoprstufhccss_yieuaABVGDEEJZIJKLMNOPRSTUFHCCSS_YIEUA___')
    tr = {ord(a):ord(b) for a, b in zip(*symbols)}
    return text.translate(tr)

## Calendars

In [5]:
html = get_content(URL_BASE, TIMEOUT, PROXIES)
soup = BeautifulSoup(html, 'lxml')
urls_clds = {}
for itm in soup.find_all('a', {'class': False, 'aria-haspopup': False}):
    if 'competitions/season' in itm['href']:
        url_ssn = URL_BASE + itm['href']
        html = get_content(url_ssn, TIMEOUT, PROXIES)
        soup_i = BeautifulSoup(html, 'lxml')
        for a in soup_i.find('div', {'class': "seasons-select"}).find_all('a'):
            urls_clds.update({
                itm.text.strip() + ' '+ a.text.strip(): URL_BASE + a['href']
            })
print(urls_clds)

{'Запад 2018-2019': 'https://www.pfl-russia.com/competitions/season-2018-2019/west/calendar/', 'Запад 2019-2020': 'https://www.pfl-russia.com/competitions/season-2019-2020/west/calendar/', 'Центр 2018-2019': 'https://www.pfl-russia.com/competitions/season-2018-2019/center/calendar/', 'Центр 2019-2020': 'https://www.pfl-russia.com/competitions/season-2019-2020/center/calendar/', 'Юг 2018-2019': 'https://www.pfl-russia.com/competitions/season-2018-2019/south/calendar/', 'Юг 2019-2020': 'https://www.pfl-russia.com/competitions/season-2019-2020/south/calendar/', 'Урал-Приволжье 2018-2019': 'https://www.pfl-russia.com/competitions/season-2018-2019/ural-privolzhe/calendar/', 'Урал-Приволжье 2019-2020': 'https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/calendar/', 'Восток 2018-2019': 'https://www.pfl-russia.com/competitions/season-2018-2019/east/calendar/', 'Восток 2019-2020': 'https://www.pfl-russia.com/competitions/season-2019-2020/east/calendar/', 'Олимп-Кубок России

In [6]:
for cld_name, url_cld in urls_clds.items():
    print('processing: ', cld_name, ' | url: ', url_cld)
    html = get_content(url_cld, TIMEOUT, PROXIES)
    file_name = '{}calendar_{}.html'.format(CACHE_PATH_CLDS_RAW, translit(cld_name))
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(str(html))
    with open(file_name, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'lxml')
    date, tour, time = '', '', ''
    cld_games = []
    game = {}
    for itm in soup.find('table', {'class': "games-table"}).find_all('tr'):
        game = {}
        if itm.find_all('td', {'class': False, 'colspan': True}):
            date = itm.find('td', {'colspan': True}).text.strip()
        if itm.find_all('td', {'class': "games-date-tr", 'colspan': True}):
            tour = itm.text.strip()
        if itm.find('span', {'class': "match-date"}):
            time = itm.find('span', {'class': "match-date"}).text.strip()
        if len(itm.find_all('td')) > 2:
            game['date'] = date
            game['tour'] = tour
            game['time'] = time
            game['team-home'] = itm.find_all('td')[1].text.strip()
            if itm.find_all('td')[1].find('a'):
                game['team-home_href'] = URL_BASE + itm.find_all('td')[1].find('a')['href']
            game['game-score'] = ' '.join(itm.find_all('td')[2].text.split())
            if itm.find_all('td')[2].find('a'):
                game['game-score_href'] = URL_BASE + itm.find_all('td')[2].find('a')['href']
            game['team-away'] = itm.find_all('td')[3].text.strip()
            if itm.find_all('td')[3].find('a'):
                game['team-away_href'] = URL_BASE + itm.find_all('td')[3].find('a')['href']
            cld_games.append(game)
    file_name = '{}calendar_{}.txt'.format(CACHE_PATH_CLDS, translit(cld_name))
    with open(file_name, 'w') as file:
        json.dump(cld_games, file)
    print('done: ', file_name)

processing:  Запад 2018-2019  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/west/calendar/
done:  ./_data/pfl/clds/calendar_Zapad_2018_2019.txt
processing:  Запад 2019-2020  | url:  https://www.pfl-russia.com/competitions/season-2019-2020/west/calendar/
done:  ./_data/pfl/clds/calendar_Zapad_2019_2020.txt
processing:  Центр 2018-2019  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/center/calendar/
done:  ./_data/pfl/clds/calendar_Centr_2018_2019.txt
processing:  Центр 2019-2020  | url:  https://www.pfl-russia.com/competitions/season-2019-2020/center/calendar/
done:  ./_data/pfl/clds/calendar_Centr_2019_2020.txt
processing:  Юг 2018-2019  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/south/calendar/
done:  ./_data/pfl/clds/calendar_Ug_2018_2019.txt
processing:  Юг 2019-2020  | url:  https://www.pfl-russia.com/competitions/season-2019-2020/south/calendar/
done:  ./_data/pfl/clds/calendar_Ug_2019_2020.txt
processing:  Урал-Приволжье 

In [7]:
for cld_name, url_cld in urls_clds.items():
    ref_name = cld_name + '_referees'
    url_ref = url_cld.replace('calendar/', 'set-ref/')
    print('processing: ', ref_name, ' | url: ', url_ref)
    html = get_content(url_ref, TIMEOUT, PROXIES)
    file_name = '{}calendar_{}.html'.format(CACHE_PATH_CLDS_RFRS_RAW, translit(ref_name))
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(str(html))
    with open(file_name, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'lxml')
    date, tour, time = '', '', ''
    rfr_games = []
    game = {}
    for itm in soup.find('table', {'class': "games-table for-judges"}).find_all('tr'):
        game = {}
        if itm.find_all('td', {'class': False, 'colspan': "8"}):
            date = itm.find('td', {'colspan': True}).text.strip()
        if itm.find_all('td', {'class': "games-date-tr", 'colspan': True}):
            tour = itm.text.strip()
        if itm.find_all('td', {'class': "header"}):
            header = []
            for td in itm.find_all('td', {'class': "header"}):
                header.append(td.text.strip())
        if len(itm.find_all('td')) > 6:
            game['date'] = date
            game['tour'] = tour
            game['header'] = header
            game['game_id'] = itm.find('span', {'class': "match-date"}).find('a').text.strip()
            game['game_id_href'] = itm.find('span', {'class': "match-date"}).find('a')['href']
            for itd, td in enumerate(itm.find_all('td', {'class': "jName"})):
                if td.find('span', {'class': "jCity"}):
                    city = td.find('span', {'class': "jCity"}).text.strip()
                else: 
                    city = ''
                game['{}_{}'.format(itd, header[itd])] = {
                    'name': td.text.replace(city, '').strip(),
                    'city': city
                }
            rfr_games.append(game)
    file_name = '{}calendar_{}.txt'.format(CACHE_PATH_CLDS_RFRS, translit(ref_name))
    with open(file_name, 'w') as file:
        json.dump(rfr_games, file)
    print('done: ', file_name)

processing:  Запад 2018-2019_referees  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/west/set-ref/
done:  ./_data/pfl/clds_rfrs/calendar_Zapad_2018_2019_referees.txt
processing:  Запад 2019-2020_referees  | url:  https://www.pfl-russia.com/competitions/season-2019-2020/west/set-ref/
done:  ./_data/pfl/clds_rfrs/calendar_Zapad_2019_2020_referees.txt
processing:  Центр 2018-2019_referees  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/center/set-ref/
done:  ./_data/pfl/clds_rfrs/calendar_Centr_2018_2019_referees.txt
processing:  Центр 2019-2020_referees  | url:  https://www.pfl-russia.com/competitions/season-2019-2020/center/set-ref/
done:  ./_data/pfl/clds_rfrs/calendar_Centr_2019_2020_referees.txt
processing:  Юг 2018-2019_referees  | url:  https://www.pfl-russia.com/competitions/season-2018-2019/south/set-ref/
done:  ./_data/pfl/clds_rfrs/calendar_Ug_2018_2019_referees.txt
processing:  Юг 2019-2020_referees  | url:  https://www.pfl-russia.com/compe

## Games

In [5]:
games_urls = []
teams_urls = []
for file_name in os.listdir(CACHE_PATH_CLDS):
    with open('{}{}'.format(CACHE_PATH_CLDS, file_name), 'r') as file:
        data = json.load(file)
    for game in data:
        for k_game, v_game in game.items():
            if 'score_href' in k_game:
                games_urls.append(v_game)
            elif '_href' in k_game:
                teams_urls.append(v_game)
            else:
                pass
print('games total: ', len(games_urls), ' | unique games: ', len(set(games_urls)))
print('teams total: ', len(teams_urls), ' | unique teams: ', len(set(teams_urls)))
games_urls = sorted(list(set(games_urls)))
teams_urls = sorted(list(set(teams_urls)))

games total:  1713  | unique games:  1713
teams total:  3282  | unique teams:  121


In [6]:
games_urls[:3]

['https://www.pfl-russia.com/competitions/season-2018-2019/center/game-36516/',
 'https://www.pfl-russia.com/competitions/season-2018-2019/center/game-36528/',
 'https://www.pfl-russia.com/competitions/season-2018-2019/center/game-36537/']

In [164]:
start_index = len(os.listdir(CACHE_PATH_GMS))
for game_url in games_urls[start_index:]:
    print('processing: ', game_url)
    html = get_content(game_url, TIMEOUT, PROXIES)
    file_name = '{}{}.html'.format(CACHE_PATH_GMS_RAW, game_url[game_url.find('/game') + 1 : -1].replace('-', '_'))
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(str(html))
    with open(file_name, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'lxml')
    game = {}
    #---overall info---
    if soup.find('div', {'class': "game-info"}).find('h3'):
        game['game_num'] = soup.find('div', {'class': "game-info"}).find('h3').text.strip()
    game['description'] = ' '.join([
        ' '.join(x.text.split()) for x in soup.find('div', {'class': "game-info"}).find_all('div', {'class': False})
    ])
    team_block = {}
    for team_num, team in enumerate(soup.find_all('div', {'class': "team-block"})):
        team_data = {}
        if team.find('a', {'class': "team-title"}):
            team_data['name'] = team.find('a', {'class': "team-title"}).text.strip()
            team_data['city'] = team.find('a', {'class': "team-city"}).text.strip()
        team_block.update({'team_' + str(team_num + 1): team_data})
    game['team_block'] = team_block
    if soup.find('div', {'class': "game-score-label"}):
        game['game_score'] = soup.find('div', {'class': "game-score-label"}).text.strip()
    #---text events---
    text_events = []
    header = []
    if soup.find('table', {'class': "text-events"}):
        for th in soup.find('table', {'class': "text-events"}).find_all('th'):
            header.append(th.text.strip())
        for tr in soup.find('table', {'class': "text-events"}).find('tbody').find_all('tr'):
            event = {}
            for itd, td in enumerate(tr.find_all('td')):
                if td.find('div', {'class': True}):
                    event_text = td.find('div')['class']
                else:
                    event_text = ' '.join(td.text.split())
                event[header[itd]] = event_text
                if td.find('a', {'href': True}):
                    event[header[itd] + '_href'] = URL_BASE + td.find('a', {'href': True})['href']
            text_events.append(event)
    game['text_events'] = text_events
    #---team players---
    teams = {}
    teams_lbl = ['teamA', 'teamB']
    players_lbl = ['main', 'reserve']
    if soup.find_all('div', {'class': "teams-tables"}):
        for lbl in teams_lbl:
            team = {}
            for imr, mr in enumerate(players_lbl):
                players = []
                header = []
                team_table = soup.find_all('div', {'class': "teams-tables"})[imr]
                for th in team_table.find('div', {'class': lbl}).find('thead').find_all('th'):
                    header.append(th.text.strip())
                for tr in team_table.find('div', {'class': lbl}).find('tbody').find_all('tr'):
                    player = {}
                    for itd, td in enumerate(tr.find_all('td')):
                        player_text = ' '.join(td.text.split())
                        player[header[itd]] = player_text
                        if td.find('div', {'class': True}):
                            for iadd, add in enumerate(td.find_all('div', {'class': True})):
                                player[header[itd] + '_' + str(iadd)] = add['class']
                        if td.find('a', {'href': True}):
                            player[header[itd] + '_href'] = URL_BASE + td.find('a', {'href': True})['href']
                    players.append(player)
                team[mr] = players
            teams[lbl] = team
    game['teams'] = teams
    #---staff---
    staff = {}
    for lbl in teams_lbl:
        coaches = {}
        staff_team = []
        header = ['title', 'name']
        if soup.find('h3', {'class': "h3 gameH3"}):
            if 'Тренер' in soup.find('h3', {'class': "h3 gameH3"}).text:
                if len(soup.find_all('div', {'class': "teams-tables"})) >= 3:
                    for tr in soup.find_all('div', {'class': "teams-tables"})[2].find('tbody').find_all('tr'):
                        staffer = {}
                        for itd, td in enumerate(tr.find_all('td')):
                            staffer.update({header[itd]: td.text.strip()})
                        staff_team.append(staffer)
            staff[lbl] = staff_team
    game['teams_staff'] = staff
    #---officials---
    officials = []
    header = ['title', 'name', 'city']
    if soup.find('table', {'class': "game-judjes table-datails"}):
        for tr in soup.find('table', {'class': "game-judjes table-datails"}).find('tbody').find_all('tr'):
            official = []
            for itd, td in enumerate(tr.find_all('td')):
                official.append({header[itd]: td.text.strip()})
            officials.append(official)
    game['officials'] = officials
    #---save to file---
    file_name = '{}{}.txt'.format(CACHE_PATH_GMS, game_url[game_url.find('/game') + 1 : -1].replace('-', '_'))
    with open(file_name, 'w') as file:
        json.dump(game, file)
    print('done: ', file_name)

processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36456/
./_data/pfl/gms_raw/game_36456.html
done:  ./_data/pfl/gms/game_36456.txt
processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36457/
./_data/pfl/gms_raw/game_36457.html
done:  ./_data/pfl/gms/game_36457.txt
processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36458/
./_data/pfl/gms_raw/game_36458.html
done:  ./_data/pfl/gms/game_36458.txt
processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36459/
./_data/pfl/gms_raw/game_36459.html
done:  ./_data/pfl/gms/game_36459.txt
processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36460/
./_data/pfl/gms_raw/game_36460.html
done:  ./_data/pfl/gms/game_36460.txt
processing:  https://www.pfl-russia.com/competitions/season-2018-2019/west/game-36461/
./_data/pfl/gms_raw/game_36461.html
done:  ./_data/pfl/gms/game_36461.txt
processing:  https://www.pfl-russi

./_data/pfl/gms_raw/game_38359.html
done:  ./_data/pfl/gms/game_38359.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38360/
./_data/pfl/gms_raw/game_38360.html
done:  ./_data/pfl/gms/game_38360.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38361/
./_data/pfl/gms_raw/game_38361.html
done:  ./_data/pfl/gms/game_38361.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38362/
./_data/pfl/gms_raw/game_38362.html
done:  ./_data/pfl/gms/game_38362.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38363/
./_data/pfl/gms_raw/game_38363.html
done:  ./_data/pfl/gms/game_38363.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38364/
./_data/pfl/gms_raw/game_38364.html
done:  ./_data/pfl/gms/game_38364.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38365/
./_data/pfl/gms_raw/ga

done:  ./_data/pfl/gms/game_38410.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38411/
./_data/pfl/gms_raw/game_38411.html
done:  ./_data/pfl/gms/game_38411.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38412/
./_data/pfl/gms_raw/game_38412.html
done:  ./_data/pfl/gms/game_38412.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38413/
./_data/pfl/gms_raw/game_38413.html
done:  ./_data/pfl/gms/game_38413.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38414/
./_data/pfl/gms_raw/game_38414.html
done:  ./_data/pfl/gms/game_38414.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38415/
./_data/pfl/gms_raw/game_38415.html
done:  ./_data/pfl/gms/game_38415.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38416/
./_data/pfl/gms_raw/game_38416.html
done:  ./_data/pfl/gms

./_data/pfl/gms_raw/game_38461.html
done:  ./_data/pfl/gms/game_38461.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38462/
./_data/pfl/gms_raw/game_38462.html
done:  ./_data/pfl/gms/game_38462.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38463/
./_data/pfl/gms_raw/game_38463.html
done:  ./_data/pfl/gms/game_38463.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38464/
./_data/pfl/gms_raw/game_38464.html
done:  ./_data/pfl/gms/game_38464.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38465/
./_data/pfl/gms_raw/game_38465.html
done:  ./_data/pfl/gms/game_38465.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38466/
./_data/pfl/gms_raw/game_38466.html
done:  ./_data/pfl/gms/game_38466.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38467/
./_data/pfl/gms_raw/ga

./_data/pfl/gms_raw/game_38512.html
done:  ./_data/pfl/gms/game_38512.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38513/
./_data/pfl/gms_raw/game_38513.html
done:  ./_data/pfl/gms/game_38513.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38514/
./_data/pfl/gms_raw/game_38514.html
done:  ./_data/pfl/gms/game_38514.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38515/
./_data/pfl/gms_raw/game_38515.html
done:  ./_data/pfl/gms/game_38515.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/center/game-38516/
./_data/pfl/gms_raw/game_38516.html
done:  ./_data/pfl/gms/game_38516.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/cup/game-37688/
./_data/pfl/gms_raw/game_37688.html
done:  ./_data/pfl/gms/game_37688.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/cup/game-37689/
./_data/pfl/gms_raw/game_376

...

./_data/pfl/gms_raw/game_38291.html
done:  ./_data/pfl/gms/game_38291.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38292/
./_data/pfl/gms_raw/game_38292.html
done:  ./_data/pfl/gms/game_38292.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38293/
./_data/pfl/gms_raw/game_38293.html
done:  ./_data/pfl/gms/game_38293.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38294/
./_data/pfl/gms_raw/game_38294.html
done:  ./_data/pfl/gms/game_38294.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38295/
./_data/pfl/gms_raw/game_38295.html
done:  ./_data/pfl/gms/game_38295.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38296/
./_data/pfl/gms_raw/game_38296.html
done:  ./_data/pfl/gms/game_38296.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/u

./_data/pfl/gms_raw/game_38340.html
done:  ./_data/pfl/gms/game_38340.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38341/
./_data/pfl/gms_raw/game_38341.html
done:  ./_data/pfl/gms/game_38341.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38342/
./_data/pfl/gms_raw/game_38342.html
done:  ./_data/pfl/gms/game_38342.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38343/
./_data/pfl/gms_raw/game_38343.html
done:  ./_data/pfl/gms/game_38343.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38344/
./_data/pfl/gms_raw/game_38344.html
done:  ./_data/pfl/gms/game_38344.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/ural-privolzhe/game-38345/
./_data/pfl/gms_raw/game_38345.html
done:  ./_data/pfl/gms/game_38345.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/u

./_data/pfl/gms_raw/game_38089.html
done:  ./_data/pfl/gms/game_38089.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38090/
./_data/pfl/gms_raw/game_38090.html
done:  ./_data/pfl/gms/game_38090.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38091/
./_data/pfl/gms_raw/game_38091.html
done:  ./_data/pfl/gms/game_38091.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38092/
./_data/pfl/gms_raw/game_38092.html
done:  ./_data/pfl/gms/game_38092.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38093/
./_data/pfl/gms_raw/game_38093.html
done:  ./_data/pfl/gms/game_38093.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38094/
./_data/pfl/gms_raw/game_38094.html
done:  ./_data/pfl/gms/game_38094.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38095/
./_data/pfl/gms_raw/game_38095.htm

./_data/pfl/gms_raw/game_38140.html
done:  ./_data/pfl/gms/game_38140.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38141/
./_data/pfl/gms_raw/game_38141.html
done:  ./_data/pfl/gms/game_38141.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38142/
./_data/pfl/gms_raw/game_38142.html
done:  ./_data/pfl/gms/game_38142.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38143/
./_data/pfl/gms_raw/game_38143.html
done:  ./_data/pfl/gms/game_38143.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38144/
./_data/pfl/gms_raw/game_38144.html
done:  ./_data/pfl/gms/game_38144.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38145/
./_data/pfl/gms_raw/game_38145.html
done:  ./_data/pfl/gms/game_38145.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38146/
./_data/pfl/gms_raw/game_38146.htm

done:  ./_data/pfl/gms/game_38191.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38192/
./_data/pfl/gms_raw/game_38192.html
done:  ./_data/pfl/gms/game_38192.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38193/
./_data/pfl/gms_raw/game_38193.html
done:  ./_data/pfl/gms/game_38193.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38194/
./_data/pfl/gms_raw/game_38194.html
done:  ./_data/pfl/gms/game_38194.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38195/
./_data/pfl/gms_raw/game_38195.html
done:  ./_data/pfl/gms/game_38195.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38196/
./_data/pfl/gms_raw/game_38196.html
done:  ./_data/pfl/gms/game_38196.txt
processing:  https://www.pfl-russia.com/competitions/season-2019-2020/west/game-38197/
./_data/pfl/gms_raw/game_38197.html
done:  ./_data/pfl/gms/game_38197.

## Teams

In [7]:
teams_urls[:3]

['https://www.pfl-russia.com/teams/10000/season-2019-2020/',
 'https://www.pfl-russia.com/teams/10001/season-2019-2020/',
 'https://www.pfl-russia.com/teams/10002/season-2019-2020/']

In [85]:
start_index = len(os.listdir(CACHE_PATH_TMS))
for team_url in teams_urls[start_index:]:
    print('processing: ', team_url)
    html = get_content(team_url, TIMEOUT, PROXIES)
    file_name = '{}team_{}.html'.format(
        CACHE_PATH_TMS_RAW, 
        team_url[team_url.find('/teams') + len('/teams') + 1 : -1].replace('-', '_').replace('/', '_')
    )
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(str(html))
    with open(file_name, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'lxml')
    team = {}
    #---overall info---
    if soup.find('h1', {'class': "club-header-name"}):
        team['club_name'] = soup.find('h1', {'class': "club-header-name"}).contents[0].strip()
        team['seasons'] = soup.find('h1', {'class': "club-header-name"}).find('div', {'class': "seasons-select"}).contents[0].strip()
        team['competitions'] = soup.find('h1', {'class': "club-header-name"}).find('a').text.strip()
        team['competitions_href'] = URL_BASE + soup.find('h1', {'class': "club-header-name"}).find('a')['href']
        for p in soup.find('div', {'class': "club-header-info"}).find_all('p'):
            if p.find('a'):
                team[p.find('span').text.strip()] = p.find('a').text.strip()
            else:
                team[p.find('span').text.strip()] = p.contents[1].strip()
    #---players---
    players = []
    if soup.find('div', {'class': "tab_team"}):
        for block in soup.find('div', {'class': "tab_team"}).find_all('div', {'class': "row article-grid-container main-players team-players"}):
            block_players = []
            for a in block.find_all('a', {'class': "article"}):
                player = {}
                block_players.append({'name': a.text.strip(), 'name_href': URL_BASE + a['href']})
            players.append({block.find('h2').text.strip(): block_players})
    team['players'] = players
    #---officials---
    if soup.find_all('table', {'class': "leadership"}):
        if len(soup.find_all('table', {'class': "leadership"})) == 2:
            officials = []
            headers = []
            for tr in soup.find_all('table', {'class': "leadership"})[0].find_all('tr'):
                if tr.find_all('th'):
                    for th in tr.find_all('th'):
                        headers.append(th['class'][0] + '_' + th.text.strip() if th.text.strip() else th['class'][0])
                if tr.find_all('td'):
                    official = {}
                    for itd, td in enumerate(tr.find_all('td')):
                        official[headers[itd]] = td.text.strip()
                    officials.append(official)
            team['officials'] = officials
            tbl_index = 1
        else:
            tbl_index = 0
    #----coaches and staff---
        coaches_and_staff = []
        headers = []
        for tr in soup.find_all('table', {'class': "leadership"})[tbl_index].find_all('tr'):
            if tr.find_all('th'):
                for th in tr.find_all('th'):
                    headers.append(th['class'][0] + '_' + th.text.strip() if th.text.strip() else th['class'][0])
            if tr.find_all('td'):
                person = {}
                for itd, td in enumerate(tr.find_all('td')):
                    person[headers[itd]] = td.text.strip()
                coaches_and_staff.append(person)
    team['coaches_and_staff'] = coaches_and_staff
    #---save to file---
    file_name = '{}team_{}.txt'.format(
        CACHE_PATH_TMS, 
        team_url[team_url.find('/teams') + len('/teams') + 1 : -1].replace('-', '_').replace('/', '_')
    )
    with open(file_name, 'w') as file:
        json.dump(team, file)
    print('done: ', file_name)

processing:  https://www.pfl-russia.com/teams/9775/season-2018-2019/
done:  ./_data/pfl/tms/team_9775_season_2018_2019.txt
processing:  https://www.pfl-russia.com/teams/9776/season-2018-2019/
done:  ./_data/pfl/tms/team_9776_season_2018_2019.txt
processing:  https://www.pfl-russia.com/teams/9776/season-2019-2020/
done:  ./_data/pfl/tms/team_9776_season_2019_2020.txt
processing:  https://www.pfl-russia.com/teams/9777/season-2018-2019/
done:  ./_data/pfl/tms/team_9777_season_2018_2019.txt
processing:  https://www.pfl-russia.com/teams/9777/season-2019-2020/
done:  ./_data/pfl/tms/team_9777_season_2019_2020.txt
processing:  https://www.pfl-russia.com/teams/9778/season-2018-2019/
done:  ./_data/pfl/tms/team_9778_season_2018_2019.txt
processing:  https://www.pfl-russia.com/teams/9778/season-2019-2020/
done:  ./_data/pfl/tms/team_9778_season_2019_2020.txt
processing:  https://www.pfl-russia.com/teams/9782/season-2018-2019/
done:  ./_data/pfl/tms/team_9782_season_2018_2019.txt
processing:  htt

## Players

In [17]:
plrs_urls = []
for file_name in os.listdir(CACHE_PATH_GMS):
    with open('{}{}'.format(CACHE_PATH_GMS, file_name), 'r') as file:
        data = json.load(file)
    for k1, v1 in data['teams'].items():
        for k2, v2 in v1.items():
            for item in v2:
                for k3, v3 in item.items():
                    if '_href' in k3:
print('players total: ', len(plrs_urls), ' | unique players: ', len(set(plrs_urls)))
plrs_urls = sorted(list(set(plrs_urls)))

players total:  47981  | unique players:  3648


In [18]:
plrs_urls[:3]

['https://www.pfl-russia.com/players/10149/season-2018-2019/',
 'https://www.pfl-russia.com/players/10149/season-2019-2020/',
 'https://www.pfl-russia.com/players/10344/season-2018-2019/']

In [113]:
start_index = len(os.listdir(CACHE_PATH_PLRS))
for plr_url in plrs_urls[start_index:]:
    print('processing: ', plr_url)
    html = get_content(plr_url, TIMEOUT, PROXIES)
    file_name = '{}player_{}.html'.format(
        CACHE_PATH_PLRS_RAW, 
        plr_url[plr_url.find('/players') + len('/players') + 1 : -1].replace('-', '_').replace('/', '_')
    )
    with open(file_name, 'w', encoding='utf-8') as file:
        file.write(str(html))
    with open(file_name, 'r', encoding='utf-8') as file:
        html = file.read()
    soup = BeautifulSoup(html, 'lxml')
    player = {}
    #---overall info---
    overall_data = soup.find('h1', {'class': "club-header-name"})
    player['name'] = overall_data.contents[0].strip()
    if overall_data.find('div', {'aria-expanded': "false"}):
        player['season'] = overall_data.find('div', {'aria-expanded': "false"}).text.strip()
    else:
        player['season'] = overall_data.find('div', {'class': "seasons-select"}).contents[0].replace('Сезон ', '').strip()
    for p in overall_data.find_all('p'):
        if p.find('a'):
            player[p.span.text.strip()] = p.find('a').text.strip()
            player[p.span.text.strip() + '_href'] = URL_BASE + p.find('a')['href']
        else:
            player[p.span.text.strip()] = p.contents[1].strip()
    #---player stats---
    plr_stats = {}
    for stat_tbl in soup.find_all('div', {'class': "player-stat-by-team"}):
        plr_stats_team = []
        headers = []
        for th in stat_tbl.find('thead').find_all('th'):
            headers.append(th.text.strip())
        for tr in stat_tbl.find('tbody').find_all('tr'):
            row = {}
            for itd, td in enumerate(tr.find_all('td')):
                row.update({headers[itd]: td.text.strip()})
                if td.find('a'):
                    row.update({headers[itd] + '_href': URL_BASE + td.find('a')['href']})
            plr_stats_team.append(row)
        if stat_tbl.find('div', {'class': "h4"}):
            plr_stats[stat_tbl.find('div', {'class': "h4"}).text.strip()] = plr_stats_team
        else:
            plr_stats['default_team'] = plr_stats_team
    player['statistics'] = plr_stats
    #---save to file---
    file_name = '{}player_{}.txt'.format(
        CACHE_PATH_PLRS, 
        plr_url[plr_url.find('/players') + len('/players') + 1 : -1].replace('-', '_').replace('/', '_')
    )
    with open(file_name, 'w') as file:
        json.dump(player, file)
    print('done: ', file_name)

processing:  https://www.pfl-russia.com/players/10149/season-2018-2019/
done:  ./_data/pfl/plrs/player_10149_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/10149/season-2019-2020/
done:  ./_data/pfl/plrs/player_10149_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/10344/season-2018-2019/
done:  ./_data/pfl/plrs/player_10344_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/10344/season-2019-2020/
done:  ./_data/pfl/plrs/player_10344_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/10349/season-2018-2019/
done:  ./_data/pfl/plrs/player_10349_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/10349/season-2019-2020/
done:  ./_data/pfl/plrs/player_10349_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/10422/season-2018-2019/
done:  ./_data/pfl/plrs/player_10422_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/10422/season-2019-2020/
done:  ./_data/pfl

done:  ./_data/pfl/plrs/player_12529_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/12645/season-2018-2019/
done:  ./_data/pfl/plrs/player_12645_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/12714/season-2018-2019/
done:  ./_data/pfl/plrs/player_12714_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/12714/season-2019-2020/
done:  ./_data/pfl/plrs/player_12714_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/12716/season-2019-2020/
done:  ./_data/pfl/plrs/player_12716_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/12791/season-2019-2020/
done:  ./_data/pfl/plrs/player_12791_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/12897/season-2018-2019/
done:  ./_data/pfl/plrs/player_12897_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/12904/season-2019-2020/
done:  ./_data/pfl/plrs/player_12904_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_13709_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/13709/season-2019-2020/
done:  ./_data/pfl/plrs/player_13709_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/13710/season-2018-2019/
done:  ./_data/pfl/plrs/player_13710_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/13710/season-2019-2020/
done:  ./_data/pfl/plrs/player_13710_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/13735/season-2018-2019/
done:  ./_data/pfl/plrs/player_13735_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/13735/season-2019-2020/
done:  ./_data/pfl/plrs/player_13735_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/13775/season-2018-2019/
done:  ./_data/pfl/plrs/player_13775_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/13775/season-2019-2020/
done:  ./_data/pfl/plrs/player_13775_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_14415_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/14471/season-2018-2019/
done:  ./_data/pfl/plrs/player_14471_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/14471/season-2019-2020/
done:  ./_data/pfl/plrs/player_14471_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/14497/season-2018-2019/
done:  ./_data/pfl/plrs/player_14497_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/14497/season-2019-2020/
done:  ./_data/pfl/plrs/player_14497_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/14518/season-2018-2019/
done:  ./_data/pfl/plrs/player_14518_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/14545/season-2018-2019/
done:  ./_data/pfl/plrs/player_14545_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/14571/season-2018-2019/
done:  ./_data/pfl/plrs/player_14571_season_2018_2019.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_15321_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/15321/season-2019-2020/
done:  ./_data/pfl/plrs/player_15321_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/15335/season-2018-2019/
done:  ./_data/pfl/plrs/player_15335_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/15335/season-2019-2020/
done:  ./_data/pfl/plrs/player_15335_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/15374/season-2018-2019/
done:  ./_data/pfl/plrs/player_15374_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/15374/season-2019-2020/
done:  ./_data/pfl/plrs/player_15374_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/15376/season-2018-2019/
done:  ./_data/pfl/plrs/player_15376_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/15389/season-2018-2019/
done:  ./_data/pfl/plrs/player_15389_season_2018_2019.txt
processing:  https://www.pfl-rus

...

done:  ./_data/pfl/plrs/player_24564_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24565/season-2019-2020/
done:  ./_data/pfl/plrs/player_24565_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24567/season-2019-2020/
done:  ./_data/pfl/plrs/player_24567_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24569/season-2019-2020/
done:  ./_data/pfl/plrs/player_24569_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24570/season-2019-2020/
done:  ./_data/pfl/plrs/player_24570_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24572/season-2019-2020/
done:  ./_data/pfl/plrs/player_24572_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24573/season-2019-2020/
done:  ./_data/pfl/plrs/player_24573_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24574/season-2019-2020/
done:  ./_data/pfl/plrs/player_24574_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_24633_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24634/season-2019-2020/
done:  ./_data/pfl/plrs/player_24634_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24635/season-2019-2020/
done:  ./_data/pfl/plrs/player_24635_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24636/season-2019-2020/
done:  ./_data/pfl/plrs/player_24636_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24637/season-2019-2020/
done:  ./_data/pfl/plrs/player_24637_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24638/season-2019-2020/
done:  ./_data/pfl/plrs/player_24638_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24639/season-2019-2020/
done:  ./_data/pfl/plrs/player_24639_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24640/season-2019-2020/
done:  ./_data/pfl/plrs/player_24640_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_24709_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24710/season-2019-2020/
done:  ./_data/pfl/plrs/player_24710_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24711/season-2019-2020/
done:  ./_data/pfl/plrs/player_24711_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24712/season-2019-2020/
done:  ./_data/pfl/plrs/player_24712_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24713/season-2019-2020/
done:  ./_data/pfl/plrs/player_24713_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24714/season-2019-2020/
done:  ./_data/pfl/plrs/player_24714_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24715/season-2019-2020/
done:  ./_data/pfl/plrs/player_24715_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24717/season-2019-2020/
done:  ./_data/pfl/plrs/player_24717_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_24780_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24781/season-2019-2020/
done:  ./_data/pfl/plrs/player_24781_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24782/season-2019-2020/
done:  ./_data/pfl/plrs/player_24782_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24783/season-2019-2020/
done:  ./_data/pfl/plrs/player_24783_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24784/season-2019-2020/
done:  ./_data/pfl/plrs/player_24784_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24785/season-2019-2020/
done:  ./_data/pfl/plrs/player_24785_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24786/season-2019-2020/
done:  ./_data/pfl/plrs/player_24786_season_2019_2020.txt
processing:  https://www.pfl-russia.com/players/24787/season-2019-2020/
done:  ./_data/pfl/plrs/player_24787_season_2019_2020.txt
processing:  https://www.pfl-rus

done:  ./_data/pfl/plrs/player_9918_season_2018_2019.txt
processing:  https://www.pfl-russia.com/players/9933/season-2018-2019/
done:  ./_data/pfl/plrs/player_9933_season_2018_2019.txt
